In [ ]:
pip install git+https://github.com/JustAnotherArchivist/snscrape.git

In [ ]:
# Get tweets using SNSCRAPE 
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
# Get 10,000 tweets containing search term: world cup within a certain period of time
query = "(world cup) lang:en until:2022-12-19_00:00:00_AST since:2022-11-20_00:00:00_AST -filter:replies"
tweets = []
limit = 10000


for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.id, tweet.username, tweet.content])

# Store tweets under a data frame        
df = pd.DataFrame(tweets, columns=['Date', 'Id', 'User', 'Tweet'])

In [ ]:
# Round1: Remove URL
df['Tweet']=df['Tweet'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
# df.head()

# Round2: Remove emoji
df['Tweet']=df['Tweet'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
# df.head()

# Round3: Remove newlines:\n
df['Tweet_processed'] = df['Tweet'].replace('\n','', regex=True)
# df.head()

# Store the filtered tweets in a new data frame
df_new=df.drop('Tweet',axis=1)
# df_new.head()

df_new.to_pickle('world_cup_tweets.pkl')

In [ ]:
pip install bertopic

In [ ]:
texts=df_new['Tweet_processed']

# Set the language to English. There is other language models as well. 
from bertopic import BERTopic
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(texts)

In [ ]:
freq = topic_model.get_topic_info(); freq.head(11)

In [ ]:
topic_model.get_topic(0)  # Select the most frequent topic, which is topic 0

In [ ]:
topic_model.get_representative_docs(0)

In [ ]:
topic_model.visualize_barchart(top_n_topics=10)